# ufunc

In [2]:
import numpy as np
np.version.version

'1.18.5'

## Reduction

* [ufunc.reduce(array, axis=0, dtype=None, out=None, keepdims=False, initial=<no value>, where=True)](https://numpy.org/doc/stable/reference/generated/numpy.ufunc.reduce.html)

### Note 
multipy & reduce may yield unexpected

In [7]:
print(np.multiply.reduce([], axis=None))

1.0


### Column-wise addition

In [2]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[5, 6, 7], [8, 9, 10]])
c = np.array([[5, 6, 7], [8, 9, 10]])
np.add.reduce((a, b, c), axis=0).tolist()

[[11, 14, 17], [20, 23, 26]]

In [3]:
a = np.array([[1,2,3], [4,5,6], [7, 8, 9]])
print(a)
np.add.reduce(a)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


array([12, 15, 18])

In [4]:
a = np.array([[1,2,3], [4,5,6], [7, 8, 9]])
b = a * 10
c = b * 10
print(a)
print(b)
print(c)
print(np.add.reduce((a, b, c)))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[10 20 30]
 [40 50 60]
 [70 80 90]]
[[100 200 300]
 [400 500 600]
 [700 800 900]]
[[111 222 333]
 [444 555 666]
 [777 888 999]]


## Accumulative application

Accumulative appliations of an ufunc on an array. Same with Scala ```def foldLeft[B](z: B)(op: (B, A) ⇒ B): B``` where ```z``` is the accumulator and ```op``` is ```ufunc```.

* [ufunc.accumulate(array, axis=0, dtype=None, out=None)](https://numpy.org/doc/stable/reference/generated/numpy.ufunc.accumulate.html)

> Accumulate the result of applying the operator to all elements.

In [5]:
a = np.array([1,2,3,4,5])
np.multiply.accumulate(a, axis=-1)

array([  1,   2,   6,  24, 120])

same with:

In [24]:
uadd = np.frompyfunc(lambda x, y: x * y, 2, 1)
uadd.accumulate([1,2,3,4,5], dtype=np.ndarray)

array([1, 2, 6, 24, 120], dtype=object)

# Custom function as ufunc
* [numpy.frompyfunc](https://numpy.org/doc/stable/reference/generated/numpy.frompyfunc.html)

> Takes an arbitrary Python function and returns a NumPy ufunc.

## frompyfunc

Note that the return type is **object** regardless with the ```dtype``` argument of ```ufunc.accumulate(array, axis=0, dtype=None, out=None)```.

In [21]:
def func(x, y) -> np.ndarray:
    return np.sin(x) * np.sin(y)

ufunc_multiply_sin = np.frompyfunc(func, 2, 1)

radians = np.array([1,2,3,4,5,6,7,8]) / 8 * np.pi
R = ufunc_multiply_sin.accumulate(radians, dtype=np.ndarray)

# frompyfunt result dtype is *object*
print(R.dtype)

# Need to cast object into the correct dtype
R.astype(float)

object


array([3.92699082e-01, 2.70598050e-01, 2.46960180e-01, 2.44457501e-01,
       2.23606555e-01, 1.56799390e-01, 5.97589517e-02, 7.31400586e-18])

Accumulative multiplication in reverse order.

In [36]:
X = np.arange(2, 14).reshape((3, 4))
print(X)

def func(x, y):
    return x**2 * y**2

ufunc_multiply = np.frompyfunc(func, 2, 1)
Z = ufunc_multiply.accumulate(X, axis=-1, dtype=np.ndarray)
Z

[[ 2  3  4  5]
 [ 6  7  8  9]
 [10 11 12 13]]


array([[2, 36, 20736, 10749542400],
       [6, 1764, 199148544, 3212471548762914816],
       [10, 12100, 21083040000, 75119583283430400000000]], dtype=object)

---

# User function

## vectorize

Note that ```vectorize``` does not create a **ufunc**, but ```vfunc```, hence ufunc operations e.g. ```accumulate```, ```reduce``` etc are not applicable.


* [Difference between frompyfunc and vectorize in numpy](https://stackoverflow.com/questions/6768245/difference-between-frompyfunc-and-vectorize-in-numpy)

> **vectorize** wraps **frompyfunc** and adds extra features:
>
> * Copies the docstring from the original function
> * Allows you to exclude an argument from broadcasting rules.
> * Returns an array of the correct dtype instead of dtype=object


* [class numpy.vectorize(pyfunc, otypes=None, doc=None, excluded=None, cache=False, signature=None)](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html)

> The vectorized function **evaluates pyfunc over *successive tuples* of the input arrays**, like the python map function

**successive tuples** menas if two arrays are given as argument to func(A, B) where A=(a0,a1,a2) and B=(b0,b1,b2), then ```pyfunc``` is applied to (a0,b0), (a1,b1), (a2, b2).

In [9]:
def func(x, y) -> np.ndarray:
    return x * y

#ufunc_multiply_sin = np.vectorize(pyfunc=func, signature="(n),(n)->(n)")
vfunc = np.vectorize(pyfunc=func)

# func(x, y) is applied on (1,-1), (2,1), (3,-1)
x = np.array([1,2,3])
y = np.array([-1,1,-1])
vfunc(x, y)

array([-1,  2, -3])